In [1]:
import gensim.models as g
import logging

In [11]:
#doc2vec parameters
vector_size = 300
window_size = 25 #15
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 200 #100
dm = 0 #0 = dbow; 1 = dmpv
worker_count = 1 #number of parallel processes

In [12]:
#pretrained word embeddings
pretrained_emb = "toy_data/pretrained_word_embeddings.txt" #None if use without pretrained embeddings

#input corpus
train_corpus = "toy_data/sinopses.txt" #train_docs

#output model
saved_path = "toy_data/model.bin"

In [13]:
#def read_corpus(fname, tokens_only=False):
    #with open(fname) as f: #
        #read_data = f.readlines()
        #for i, line in enumerate(read_data):
            #t_corpus = line.split(" | ")
            #if(len(t_corpus)==2):
                # For training data, add tags
                #yield g.doc2vec.TaggedDocument(t_corpus[1], [i])
            #yield g.doc2vec.TaggedDocument(line, [i])
#import nltk
#from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
            
def read_corpus(fname, tokens_only=False): 
    with open(fname) as f: #
        read_data = f.readlines()
        for i, line in enumerate(read_data):
            t_corpus = line.split(" | ")
            if(len(t_corpus)==2):
                #corpus_tokenized = word_tokenize(t_corpus[1])
                #text = nltk.Text(corpus_tokenized)
                #print('text: ', text)
                #removing stopwords
                #filtered_words = [word for word in corpus_tokenized if word not in stopwords.words('portuguese')]
                #print(filtered_words)
                # For training data, add tags
                #train_docs =  list(g.doc2vec.TaggedDocument(t_corpus[1], [i]))
                yield g.doc2vec.TaggedDocument(t_corpus[1], [i])

In [14]:
train_docs = list(read_corpus(train_corpus))

In [15]:
train_docs[:5]

[TaggedDocument(words='As coisas estavam mal para a Inteligência Britânica, pois a Smersh começara a sabotar a estabilidade global: nada menos do que onze agentes foram abatidos e, para piorar as coisas, seu maior agente secreto, o 007, estava desfrutando de sua aposentadoria. Sir James Bond, o primeiro 007, é convencido por alguns chefes de agências de espionagem a combater o inimigo comum. Essa versão de "Cassino Royale" não é uma versão oficial dos filmes de 007, pois foi rodado por outra equipe, estúdios, fora dos padrões e contratos. É uma produção anglo-americana de 1967, do gênero comédia de espionagem.\n', tags=[0]),
 TaggedDocument(words='A ação é eletrizante e ininterrupta, pois o agente 007 (Sean Connery) vai além do dever do ofício e até as profundezas do oceano para encontrar um perigoso criminoso que está ameaçando milhões de pessoas através da chantagem de destruir o mundo por meio de um holocausto nuclear.\n', tags=[1]),
 TaggedDocument(words='Com a queda da Cortina de 

In [16]:
#enable logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [17]:
model = g.doc2vec.Doc2Vec(size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=worker_count, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, iter=train_epoch)

In [18]:
model.build_vocab(train_docs)

2018-01-24 18:37:41,455 : INFO : collecting all words and their counts
2018-01-24 18:37:41,461 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-01-24 18:37:41,471 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-01-24 18:37:41,751 : INFO : collected 157 word types and 7771 unique tags from a corpus of 7529 examples and 2793992 words
2018-01-24 18:37:41,751 : INFO : Loading a fresh vocabulary
2018-01-24 18:37:41,752 : INFO : min_count=1 retains 157 unique words (100% of original 157, drops 0)
2018-01-24 18:37:41,753 : INFO : min_count=1 leaves 2793992 word corpus (100% of original 2793992, drops 0)
2018-01-24 18:37:41,754 : INFO : deleting the raw counts dictionary of 157 items
2018-01-24 18:37:41,755 : INFO : sample=1e-05 downsamples 94 most-common words
2018-01-24 18:37:41,756 : INFO : downsampling leaves estimated 57975 word corpus (2.1% of prior 2793992)
2018-01-2

In [19]:
#train doc2vec model
%time model.train(train_docs, total_examples=model.corpus_count, epochs=model.iter)

2018-01-24 18:37:54,620 : INFO : training model with 1 workers on 158 vocabulary and 300 features, using sg=1 hs=0 sample=1e-05 negative=5 window=25
2018-01-24 18:37:55,649 : INFO : PROGRESS: at 0.62% examples, 81369 words/s, in_qsize 1, out_qsize 0
2018-01-24 18:37:56,649 : INFO : PROGRESS: at 1.32% examples, 86446 words/s, in_qsize 2, out_qsize 0
2018-01-24 18:37:57,651 : INFO : PROGRESS: at 2.02% examples, 88114 words/s, in_qsize 2, out_qsize 0
2018-01-24 18:37:58,653 : INFO : PROGRESS: at 2.72% examples, 89059 words/s, in_qsize 1, out_qsize 0
2018-01-24 18:37:59,655 : INFO : PROGRESS: at 3.36% examples, 88014 words/s, in_qsize 1, out_qsize 0
2018-01-24 18:38:00,657 : INFO : PROGRESS: at 4.07% examples, 88733 words/s, in_qsize 2, out_qsize 0
2018-01-24 18:38:01,658 : INFO : PROGRESS: at 4.79% examples, 89372 words/s, in_qsize 1, out_qsize 0
2018-01-24 18:38:02,660 : INFO : PROGRESS: at 5.49% examples, 89753 words/s, in_qsize 1, out_qsize 0
2018-01-24 18:38:03,662 : INFO : PROGRESS: 

CPU times: user 2min 28s, sys: 1.24 s, total: 2min 29s
Wall time: 2min 22s


13097163

In [20]:
#save model
model.save(saved_path)

2018-01-24 18:42:06,655 : INFO : saving Doc2Vec object under toy_data/model.bin, separately None
2018-01-24 18:42:06,663 : INFO : not storing attribute syn0norm
2018-01-24 18:42:06,670 : INFO : not storing attribute cum_table
2018-01-24 18:42:06,811 : INFO : saved toy_data/model.bin
